### Example in UWG
2_15_Rayleigh-Taylor_van_Keken_et_al_1997

### Ref:
Kaus, B. J., Mühlhaus, H., & May, D. A. (2010). A stabilization algorithm for geodynamic numerical simulations with a free surface. Physics of the Earth and Planetary Interiors, 181(1-2), 12-20.

In [ ]:
import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis

u = GEO.UnitRegistry

In [ ]:
Res_mesh = (50,50)  # res = 5km
Res_real = [(-250.,250.),(-500,0.)] # Unit: km 
dt = 500*u.year
fdir = "Kaus2010_RT_0.5ka_rs"

In [ ]:
# Scaling
# test 3
# dRho =   100. * u.kilogram / u.meter**3 # matprop.ref_density
# g    =   9.81 * u.meter / u.second**2   # modprop.gravity
boxLength = 500. * u.kilometer #  modprop.boxHeight

# # lithostatic pressure for mass-time-length
# ref_stress = dRho * g * boxLength

# ref_viscosity = 1e21 * u.pascal * u.second
# KL = boxLength
# KT = ref_viscosity/ref_stress
# KM = ref_viscosity*KL*KT 

# GEO.scaling_coefficients["[time]"] = KT
# GEO.scaling_coefficients["[length]"] = KL
# GEO.scaling_coefficients["[mass]"] = KM

half_rate = 1.0 * u.centimeter / u.year
model_length = 500. * u.kilometer
bodyforce = 3200 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [ ]:
Model = GEO.Model(elementRes=Res_mesh,
                  minCoord=(Res_real[0][0] * u.kilometer, Res_real[1][0] * u.kilometer),  
                  maxCoord=(Res_real[0][1] * u.kilometer, Res_real[1][1] * u.kilometer),
                  gravity=(0.0, -9.81 * u.meter / u.second**2))

Model.outputDir = fdir

In [ ]:
import math
import underworld.function as fn

# material perturbation from van Keken et al. 1997
wavelength = GEO.nd(boxLength)
amplitude  = GEO.nd(5*u.kilometer)
offset     = GEO.nd(-100.*u.kilometer)
k = 2. * math.pi / wavelength

# Create function to return particle's coordinate
coord = fn.coord()

# Define the material perturbation, a function of the x coordinate (accessed by `coord[0]`).
perturbationFn = offset + amplitude*fn.math.cos(k*coord[0])

lightShape = perturbationFn > coord[1] 
densShape = perturbationFn < coord[1]

In [ ]:
densMaterial = Model.add_material(name="Dense Material", shape=densShape)
lightMaterial = Model.add_material(name="Light Material", shape=lightShape)

In [ ]:
densMaterial.density  = 3300 * u.kilogram / u.metre**3
lightMaterial.density = 3200 * u.kilogram / u.metre**3

In [ ]:
densMaterial.viscosity = 1e21 * u.pascal * u.second
lightMaterial.viscosity = 1e20 * u.pascal * u.second

In [ ]:
Fig = vis.Figure(figsize=(500, 500))
Fig.Points(Model.swarm, Model.materialField, fn_size=3.0)
Fig.save("F_Kaus2010_RT_material.png")
Fig.show()

In [ ]:
Model.set_velocityBCs(left=[0, None], right=[0., None],top=[None,None],bottom=[0.,0.])

In [ ]:
Model.freesurface = True

In [ ]:
#Model.init_model(temperature=False, pressureField=True)

In [ ]:
Model.run_for(2.0 * u.megayears, checkpoint_interval=0.1*u.megayears,dt= dt)

In [ ]:
Fig = vis.Figure(figsize=(500, 500))
Fig.Surface(Model.mesh, Model.velocityField[1])
Fig.VectorArrows(Model.mesh, Model.velocityField)
Fig.save("Figure_2.png")
Fig.show()

In [ ]:
Fig = vis.Figure(figsize=(500, 500))
Fig.Points(Model.swarm, Model.materialField, fn_size=3.0)
Fig.VectorArrows(Model.mesh, Model.velocityField)
Fig.save("Figure_3.png")
Fig.show()

In [ ]:
Fig = vis.Figure(figsize=(500, 500))
Fig.Mesh(Model.mesh, fn_size=3.0)
#Fig.VectorArrows(Model.mesh, Model.velocityField)
Fig.save("Figure_mesh_3.png")
Fig.show()

In [ ]:
Model.mesh.data